1. Скачиваем все нужные библиотеки

In [1]:
import requests

In [2]:
import bs4

In [3]:
from bs4 import BeautifulSoup

In [4]:
import time
from datetime import datetime
import random

In [5]:
from pymystem3 import Mystem

In [6]:
session = requests.session()

2. Функция для выгрузки отзывов с сайта

In [8]:
def feedbacks_loading(link):
    positive_list = []
    negative_list = []
    response = session.get(link)
    page = response.text
    soup = BeautifulSoup(page, 'html.parser')
    ratings = soup.find_all('span', {'class': 'com-star'})
    feedbacks_body = soup.find_all('div', {'class': 'text', 'itemprop': 'description'})
    for i in range(len(ratings)):
        stroka = ''
        rating = str(ratings[i])
        rate = int(rating[rating.find('meta') + 14])
        if rate > 4:
            positive_list.append(feedbacks_body[i].text)
        elif rate <= 3:
            negative_list.append(feedbacks_body[i].text)
    return positive_list, negative_list

In [9]:
pos_list, neg_list = feedbacks_loading('https://www.kaluga-poisk.ru/catalog/objects/peppers-pizza-peppers-pitstsa-y/reviews#cm-227550')

In [10]:
print(len(pos_list), len(neg_list))

183 30


3. Обрабатываем отзывы с помощью pymystem

In [11]:
positive_set = set()
negative_set = set()

In [12]:
m = Mystem()

In [13]:
for text in neg_list:
    lemmas = m.lemmatize(text)
    text_set = set(lemmas)
    negative_set = negative_set | text_set

In [14]:
k = 0
for text in pos_list[::3]:
    k += 1
    lemmas = m.lemmatize(text)
    text_set = set(lemmas)
    positive_set = positive_set | text_set
    if k == 30:
        break

In [15]:
only_positive = positive_set - negative_set
only_negative = negative_set - positive_set

In [16]:
print(len(positive_set), len(negative_set))
print(len(only_positive), len(only_negative))

459 806
225 572


4. Скачиваем тестовые отзывы (они о другой пиццерии)

In [17]:
test_positive, test_negative = feedbacks_loading('https://www.kaluga-poisk.ru/catalog/objects/paprika-kaluga/reviews#cm-20978')

In [18]:
print(len(test_positive), len(test_negative))

9 8


5. Функция, которая возвращает тон отзыва и считает вероятности того, будет ли он положительным или отрицательным

In [19]:
def check_tone(text, positive_set, negative_set):
    prediction = 0
    probability = [0, 0]
    lemmas = m.lemmatize(text)
    text_set = set(lemmas)
    positive_len = len(text_set & positive_set)
    negative_len = len(text_set & negative_set)
    if negative_len > positive_len:
        prediction = -1
    else:
        prediction = 1
    if positive_len + negative_len != 0:
        probability = [positive_len / (positive_len + negative_len), negative_len / (positive_len + negative_len)]
    return prediction, probability

6. Считаем, как хорошо работает функция на тестовых данных

In [20]:
right_answers = 0
all_answers = 0
print('positive')
for text in test_positive:
    pred, prob = check_tone(text, only_positive, only_negative)
    print(prob)
    if pred == 1:
        right_answers += 1
    all_answers += 1
print('negative')
for text in test_negative:
    pred, prob = check_tone(text, only_positive, only_negative)
    print(prob)
    if pred == -1:
        right_answers += 1
    all_answers += 1
print('accuracy_score =', right_answers / all_answers)

positive
[0.6, 0.4]
[0.42857142857142855, 0.5714285714285714]
[0.42105263157894735, 0.5789473684210527]
[0.7142857142857143, 0.2857142857142857]
[0.625, 0.375]
[0.0, 1.0]
[0.3333333333333333, 0.6666666666666666]
[0.29545454545454547, 0.7045454545454546]
[0.3076923076923077, 0.6923076923076923]
negative
[0.27586206896551724, 0.7241379310344828]
[0.0, 1.0]
[0.2962962962962963, 0.7037037037037037]
[0.4444444444444444, 0.5555555555555556]
[0.13636363636363635, 0.8636363636363636]
[0.18181818181818182, 0.8181818181818182]
[0.17857142857142858, 0.8214285714285714]
[0.2222222222222222, 0.7777777777777778]
accuracy_score = 0.6470588235294118


Тест хорошо предсказывает негативные и плохо позитивные отзывы. Почему? Можно посмотреть на размер множества "позитивных" и "негативных" слов. Второе множество значительно больше. Попробуем сделать вес "положительных" слов больше.

In [21]:
coef = len(only_positive) / len(only_negative)

In [22]:
def new_check_tone(text, positive_set, negative_set, coef):
    prediction = 0
    probability = [0, 0]
    lemmas = m.lemmatize(text)
    text_set = set(lemmas)
    positive_len = len(text_set & positive_set)
    negative_len = len(text_set & negative_set) * coef
    if negative_len > positive_len:
        prediction = -1
    else:
        prediction = 1
    if positive_len + negative_len != 0:
        probability = [positive_len / (positive_len + negative_len), negative_len / (positive_len + negative_len)]
    return prediction, probability

In [24]:
right_answers = 0
all_answers = 0
print('positive')
for text in test_positive:
    pred, prob = new_check_tone(text, only_positive, only_negative, coef)
    print(prob)
    if pred == 1:
        right_answers += 1
    all_answers += 1
print('negative')
for text in test_negative:
    pred, prob = new_check_tone(text, only_positive, only_negative, coef)
    print(prob)
    if pred == -1:
        right_answers += 1
    all_answers += 1
print('accuracy_score =', right_answers / all_answers)

positive
[0.7922437673130194, 0.2077562326869806]
[0.6559633027522936, 0.3440366972477064]
[0.6489859594383776, 0.35101404056162244]
[0.8640483383685801, 0.13595166163141995]
[0.809052333804809, 0.19094766619519093]
[0.0, 1.0]
[0.5596868884540117, 0.44031311154598823]
[0.5159947262507807, 0.4840052737492193]
[0.5304892186413169, 0.469510781358683]
negative
[0.49199010859047415, 0.5080098914095258]
[0.0, 1.0]
[0.517003728392272, 0.4829962716077279]
[0.6703779665983006, 0.3296220334016994]
[0.28642964446670005, 0.7135703555333]
[0.36099715998737775, 0.6390028400126223]
[0.35594275046670815, 0.6440572495332918]
[0.4207429201912468, 0.5792570798087532]
accuracy_score = 0.8235294117647058


# Мы пытались...
И у нас что-то получилось! Но модель стала менее уверена. Вероятности на негативных отзывах стали ближе к 0.5.

В положительных отзывах часто используются более нейтральные слова. Можно попробовать для балланса множеств брать слова не из 30 положительных отзывов, а из большей части, найденных на одной странице.

In [28]:
k = 0
new_positive_set = set()
for text in pos_list[::2]:
    k += 1
    lemmas = m.lemmatize(text)
    text_set = set(lemmas)
    new_positive_set = new_positive_set | text_set
    if k == 70:
        break

In [29]:
new_only_positive = new_positive_set - negative_set
new_only_negative = negative_set - new_positive_set

In [30]:
print(len(new_positive_set), len(negative_set))
print(len(new_only_positive), len(new_only_negative))

824 806
483 465


получилось почти поровну

In [31]:
right_answers = 0
all_answers = 0
print('positive')
for text in test_positive:
    pred, prob = check_tone(text, only_positive, only_negative)
    print(prob)
    if pred == 1:
        right_answers += 1
    all_answers += 1
print('negative')
for text in test_negative:
    pred, prob = check_tone(text, only_positive, only_negative)
    print(prob)
    if pred == -1:
        right_answers += 1
    all_answers += 1
print('accuracy_score =', right_answers / all_answers)

positive
[0.6, 0.4]
[0.42857142857142855, 0.5714285714285714]
[0.42105263157894735, 0.5789473684210527]
[0.7142857142857143, 0.2857142857142857]
[0.625, 0.375]
[0.0, 1.0]
[0.3333333333333333, 0.6666666666666666]
[0.29545454545454547, 0.7045454545454546]
[0.3076923076923077, 0.6923076923076923]
negative
[0.27586206896551724, 0.7241379310344828]
[0.0, 1.0]
[0.2962962962962963, 0.7037037037037037]
[0.4444444444444444, 0.5555555555555556]
[0.13636363636363635, 0.8636363636363636]
[0.18181818181818182, 0.8181818181818182]
[0.17857142857142858, 0.8214285714285714]
[0.2222222222222222, 0.7777777777777778]
accuracy_score = 0.6470588235294118


# Мы пытались...
И у нас не получилось(

Вообще я выбрала порог вхождения в отрицательный отзыв довольно большой - 3 звезды (из 5 и одной суперзвезды - странный сайт). Попробуем его уменьшить, но нам понадобится больше страничек.

In [34]:
def new_feedbacks_loading(link):
    positive_list = []
    negative_list = []
    response = session.get(link)
    page = response.text
    soup = BeautifulSoup(page, 'html.parser')
    ratings = soup.find_all('span', {'class': 'com-star'})
    feedbacks_body = soup.find_all('div', {'class': 'text', 'itemprop': 'description'})
    for i in range(len(ratings)):
        stroka = ''
        rating = str(ratings[i])
        rate = int(rating[rating.find('meta') + 14])
        if rate > 4:
            positive_list.append(feedbacks_body[i].text)
        elif rate <= 2:
            negative_list.append(feedbacks_body[i].text)
    return positive_list, negative_list

In [35]:
new_pos_list, new_neg_list = new_feedbacks_loading('https://www.kaluga-poisk.ru/catalog/objects/peppers-pizza-peppers-pitstsa-y/reviews#cm-227550')

In [36]:
print(len(new_pos_list), len(new_neg_list))

183 20


In [37]:
new_pos_list_1, new_neg_list_1 = new_feedbacks_loading('https://www.kaluga-poisk.ru/catalog/objects/tomato-kaluga/reviews#cm-30789')

In [38]:
print(len(new_pos_list_1), len(new_neg_list_1))

29 22


In [39]:
final_pos_list = new_pos_list + new_pos_list_1
final_neg_list = new_neg_list + new_neg_list_1

Получилось 42 негативных отзыва. Попробуем взять по 42 позитивных и негативных.

In [40]:
positive_set_2 = set()
negative_set_2 = set()

In [44]:
for text in neg_list:
    lemmas = m.lemmatize(text)
    text_set = set(lemmas)
    negative_set_2 = negative_set_2 | text_set
k = 0
for text in pos_list[::3]:
    k += 1
    lemmas = m.lemmatize(text)
    text_set = set(lemmas)
    positive_set_2 = positive_set_2 | text_set
    if k == 42:
        break

In [45]:
only_positive_2 = positive_set_2 - negative_set_2
only_negative_2 = negative_set_2 - positive_set_2

In [46]:
print(len(positive_set_2), len(negative_set_2))
print(len(only_positive_2), len(only_negative_2))

573 806
306 539


перевес "негативных" слов чтал чуть меньше

In [47]:
right_answers = 0
all_answers = 0
print('positive')
for text in test_positive:
    pred, prob = check_tone(text, only_positive_2, only_negative_2)
    print(prob)
    if pred == 1:
        right_answers += 1
    all_answers += 1
print('negative')
for text in test_negative:
    pred, prob = check_tone(text, only_positive_2, only_negative_2)
    print(prob)
    if pred == -1:
        right_answers += 1
    all_answers += 1
print('accuracy_score =', right_answers / all_answers)

positive
[0.75, 0.25]
[0.6, 0.4]
[0.47058823529411764, 0.5294117647058824]
[0.7142857142857143, 0.2857142857142857]
[0.75, 0.25]
[0.0, 1.0]
[0.3333333333333333, 0.6666666666666666]
[0.35714285714285715, 0.6428571428571429]
[0.375, 0.625]
negative
[0.3103448275862069, 0.6896551724137931]
[0.0, 1.0]
[0.35714285714285715, 0.6428571428571429]
[0.4444444444444444, 0.5555555555555556]
[0.22727272727272727, 0.7727272727272727]
[0.2, 0.8]
[0.18518518518518517, 0.8148148148148148]
[0.25, 0.75]
accuracy_score = 0.7058823529411765


# Мы пытались...
и снова неплохо получилось!





# Дальше идёт всякого рода помойка и попытки работы с сайтами irecommend.ru и Кинопоиск

In [113]:
coef_1 = len(new_only_positive) / len(new_only_negative)

In [114]:
right_answers = 0
all_answers = 0
print('positive')
for text in test_positive:
    pred, prob = new_check_tone(text, only_positive, only_negative, coef_1)
    print(prob)
    if pred == 1:
        right_answers += 1
    all_answers += 1
print('negative')
for text in test_negative:
    pred, prob = new_check_tone(text, only_positive, only_negative, coef_1)
    print(prob)
    if pred == -1:
        right_answers += 1
    all_answers += 1
print('accuracy_score =', right_answers / all_answers)

positive
[0.10997442455242966, 0.8900255754475703]
[0.0899581589958159, 0.9100418410041841]
[0.17077045274027006, 0.82922954725973]
[0.2478386167146974, 0.7521613832853026]
[0.2833607907742998, 0.7166392092257001]
[0.0, 1.0]
[0.10997442455242966, 0.8900255754475703]
[0.1281148429035753, 0.8718851570964247]
[0.0970216606498195, 0.9029783393501805]
negative
[0.07915324436263231, 0.9208467556373677]
[0.0, 1.0]
[0.09335649153278333, 0.9066435084672166]
[0.22872340425531915, 0.7712765957446809]
[0.043000000000000003, 0.957]
[0.0899581589958159, 0.9100418410041841]
[0.08245445829338448, 0.9175455417066155]
[0.0723905723905724, 0.9276094276094277]
[0.058186738836265225, 0.9418132611637348]
[0.031899109792284865, 0.9681008902077152]
accuracy_score = 0.5263157894736842


Объединение двух вариантов не срабатывает, так как негативные слова в любом случае чаще повторяются, чем нейтральные и положительные. А так как соотношение положительных и отрицательных теперь стало равным 2:1, то мы только больше увеличиваем вероятность того, что отзыв отрицательный.

In [37]:
for elem in a:
    stroka += str(elem)
    rate = int(stroka[stroka.find('meta') + 14])
print(a)
print(rate)

<span class="com-star" itemprop="reviewRating" itemscope="" itemtype="https://schema.org/Rating">
<meta content="3" itemprop="ratingValue"/>
<meta content="1" itemprop="worstRating"/>
<meta content="6" itemprop="bestRating"/>
</span>
3


In [ ]:
for i in range(2, 101):
    link = base_link + str(i)
    response = session.get(link)
    page = response.text
    soup = BeautifulSoup(page, 'html.parser')
    feedbacks = soup.find_all('div', {'class': 'reviewTextSnippet'})
    for feedback in feedbacks:
        feedback_link = feedback.find('a')
        href_feedback = feedback_link.attrs['href']
        response_1 = session.get(base_feedback + href_feedback)
        page_1 = response_1.text
        soup_1 = BeautifulSoup(page_1, 'html.parser')
        try:
            feedback_text = soup_1.find('div', {'class': 'description hasinlineimage', 'itemprop':'reviewBody'}).text
            stars = soup_1.find_all('div', {'class': 'on'})
            feedback_text = feedback_text.replace("\n", " ")
            if len(stars) / 2 > 3:
                print("+++")
                f_positive.write(feedback_text)
                f_positive.write("\n")
            elif len(stars) / 2 < 3:
                print("---")
                f_negative.write(feedback_text)
                f_negative.write("\n")
        except Exception as e:
            print(e)
        time.sleep(random.uniform(1.1, 5.2))
    time.sleep(random.uniform(1.1, 5.2))

In [7]:
base_link = "https://irecommend.ru/catalog/reviews/5?page="
base_feedback = "https://irecommend.ru"

In [12]:
known_proxy_ip = '84.247.50.78:8443'
proxy = {'http': known_proxy_ip, 'https': known_proxy_ip}

In [10]:
f_positive = open("positive_feedback.txt", "a")
f_negative = open("negative_feedback.txt", "a")

In [14]:
for i in range(2, 101):
    link = base_link + str(i)
    response = session.get(link)
    page = response.text
    soup = BeautifulSoup(page, 'html.parser')
    feedbacks = soup.find_all('div', {'class': 'reviewTextSnippet'})
    for feedback in feedbacks:
        feedback_link = feedback.find('a')
        href_feedback = feedback_link.attrs['href']
        response_1 = session.get(base_feedback + href_feedback)
        page_1 = response_1.text
        soup_1 = BeautifulSoup(page_1, 'html.parser')
        try:
            feedback_text = soup_1.find('div', {'class': 'description hasinlineimage', 'itemprop':'reviewBody'}).text
            stars = soup_1.find_all('div', {'class': 'on'})
            feedback_text = feedback_text.replace("\n", " ")
            if len(stars) / 2 > 3:
                print("+++")
                f_positive.write(feedback_text)
                f_positive.write("\n")
            elif len(stars) / 2 < 3:
                print("---")
                f_negative.write(feedback_text)
                f_negative.write("\n")
        except Exception as e:
            print(e)
        time.sleep(random.uniform(1.1, 5.2))
    time.sleep(random.uniform(1.1, 5.2))

+++
+++
+++
+++
+++
+++
+++
+++
+++
+++
+++
+++
+++
+++
+++
+++
+++
'NoneType' object has no attribute 'text'


In [15]:
f_positive.close()
f_negative.close()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [52]:
stars = soup_1.find_all('div', {'class': 'on'})

In [54]:
len(stars)

8

In [47]:
feedback_set = set(nltk.word_tokenize(feedback_text))


🔸🔸🔸Описание продукта🔸🔸🔸Платежный счет - это обычный счет, которым вы можете пользоваться, он является отдельным продуктом с определенными возможностями и исключениями. Почти как карта, ПОЧТИ!Как открыть: на главном экране в разделе «Кошелёк» нажмите «+», далее выбираете «Платёжный счёт». Он появляется моментально.Можно открыть неограниченное число счетов.При использовании можно в дальнейшем открыть банковскую карту и прикрепить ее к действующему счету, а не открывать совместно с картой и новый счет.Обслуживание: бесплатно, без каких-либо условий.Кэшбек: начисляют баллы точно так же как и по карте, исключений нет.Информация о счете от банка: 










 










Эта вся информация от банка по поводу пользования платежным счетом. В теории не все моменты понятны и только при пользовании счетом можно столкнуться с тем, где именно разница между платежным счетом и банковской картой, и какие ограничения действительно есть.И что же я выяснила? Рассказываю.🔸🔸🔸Мой отзыв🔸🔸🔸Я открыла себе Плате

In [ ]:
<div class="smTeaser plate teaser-item " data-type="1" data-nid="9431363"
<div class="reviewTextSnippet" data-text="Читать весь отзыв">
class="more"

Посчитать количество on-off в stars rating